# LLMCheckerChain
This notebook showcases how to use LLMCheckerChain.

In [1]:
from langchain.chains import LLMCheckerChain, LLMChain, SequentialChain, ParallelChain
from langchain.llms import OpenAI
from langchain.prompts.base import ListOutputParser
from langchain.llms.base import LLM


llm = OpenAI(temperature=0.7)

text = "What type of mammal lays the biggest eggs?"

checker_chain = LLMCheckerChain(llm=llm, verbose=True)

# checker_chain.run(text)

In [2]:
# flake8: noqa
from langchain.prompts.prompt import PromptTemplate

_CREATE_DRAFT_ANSWER_TEMPLATE = """{question}\n\n"""
CREATE_DRAFT_ANSWER_PROMPT = PromptTemplate(
    input_variables=["question"], template=_CREATE_DRAFT_ANSWER_TEMPLATE
)

_LIST_ASSERTIONS_TEMPLATE = """Here is a statement:
{statement}
Make a bullet point list of the assumptions you made when producing the above statement.\n\n"""
LIST_ASSERTIONS_PROMPT = PromptTemplate(
    input_variables=["statement"], template=_LIST_ASSERTIONS_TEMPLATE
)

_CHECK_ASSERTIONS_TEMPLATE = """{assertions}
Repeat the assertion above, and then determine if it is true or false. If it is false, explain why.\n\n"""
CHECK_ASSERTIONS_PROMPT = PromptTemplate(
    input_variables=["assertions"], template=_CHECK_ASSERTIONS_TEMPLATE
)

_REVISED_ANSWER_TEMPLATE = """{checked_assertions}

Question: In light of the above assertions and checks, how would you answer the question '{question}'?

The answer is: """
REVISED_ANSWER_PROMPT = PromptTemplate(
    input_variables=["checked_assertions", "question"],
    template=_REVISED_ANSWER_TEMPLATE,
)

In [3]:
llm = OpenAI(temperature=0.7)

create_draft_answer_chain = ParallelChain(
    llm=llm, prompt=CREATE_DRAFT_ANSWER_PROMPT, output_key="statement"
)

list_assertions_chain = ParallelChain(
    llm=llm,
    prompt=LIST_ASSERTIONS_PROMPT,
    output_key="assertions",
    # This indicates that there are multiple outputs
    output_parser=lambda x: [i.strip() for i in x.split("\n") if len(i.strip()) > 0],
)

check_assertions_chain = ParallelChain(
    llm=llm, prompt=CHECK_ASSERTIONS_PROMPT, output_key="checked_assertions"
)

revised_answer_chain = ParallelChain(
    llm=llm,
    prompt=REVISED_ANSWER_PROMPT,
    output_key="revised_answer",
    # This indicates that there are multiple inputs
    input_parser=lambda x: "\n".join(x),
)

self_ask_verification_chain = SequentialChain(
    chains=[
        create_draft_answer_chain,
        list_assertions_chain,
        check_assertions_chain,
        revised_answer_chain,
    ],
    input_variables=["question"],
    output_variables=["revised_answer"],
    verbose=True,
)

In [4]:
# %%time
# self_ask_verification_chain.run(text)

In [18]:
llm = OpenAI(temperature=0.7)

create_draft_answer_chain = ParallelChain(
    llm=llm, prompt=CREATE_DRAFT_ANSWER_PROMPT, output_key="statement", n = 25
)

list_assertions_chain = ParallelChain(
    llm=llm,
    prompt=LIST_ASSERTIONS_PROMPT,
    output_key="assertions",
    # This indicates that there are multiple outputs
#     output_parser=lambda x: [i.strip() for i in x.split("\n") if len(i.strip()) > 0],
)

check_assertions_chain = ParallelChain(
    llm=llm, prompt=CHECK_ASSERTIONS_PROMPT, output_key="checked_assertions"
)

revised_answer_chain = ParallelChain(
    llm=llm,
    prompt=REVISED_ANSWER_PROMPT,
    output_key="revised_answer",
    # This indicates that there are multiple inputs
#     input_parser=lambda x: "\n".join(x),
)


MOE_CONSENSUS_PROMPT = PromptTemplate(
    input_variables=["revised_answer", "question"],
    template="""{revised_answer}

Question: A group of people each gave their answers above. Which answer best answers the question '{question}'?

Answer: """,
)

moe_answer_chain = ParallelChain(
    llm=llm,
    prompt=MOE_CONSENSUS_PROMPT,
    output_key="final_answer",
    # This indicates that there are multiple inputs
    input_parser=lambda x: "\n -" + "\n -".join(x),
)


self_ask_verification_chain = SequentialChain(
    chains=[
        create_draft_answer_chain,
        list_assertions_chain,
        check_assertions_chain,
        revised_answer_chain,
        moe_answer_chain,
    ],
    input_variables=["question"],
    output_variables=["final_answer"],
    verbose=True,
)

In [19]:
%%time
self_ask_verification_chain.run("What type of mammal lays the biggest eggs?")



> Entering new SequentialChain chain...
Chain 0:
{'statement': ['\n\nThe largest mammal that lays eggs is the Long-Beaked Echidna. This species is native to parts of New Guinea and Australia, and its eggs can reach up to 4 inches in length.', '\n\nThe largest egg laid by a mammal belongs to the Short-beaked Echidna, a spiny anteater found in Australia and New Guinea. The eggs are about the size of a tennis ball, with a diameter of about 4.5 cm (1.7 in).', '\n\nThe biggest mammal eggs are laid by the elephant, which can lay eggs up to 12 inches long and weigh up to 2.2 pounds.', '\nThe largest mammal that lays eggs is the platypus.', '\n\nThe largest mammal that lays eggs is the echidna, also known as the spiny anteater.', '\n\nThe largest mammal that lays eggs is the Australian platypus, which lays eggs up to 1.5 inches (3.8 cm) in diameter.', '\nA platypus is the mammal that lays the biggest eggs.', '\nNo mammal lays eggs. Mammals give birth to live young.', '\nThe largest mammal eg

Chain 3:
{'revised_answer': [' The Long-Beaked Echidna is the mammal that lays the biggest eggs.', '\n\nThe type of mammal that lays the biggest eggs is the Ostrich, which can lay eggs that are up to 15 cm (5.9 in) in diameter.', ' No mammal lays eggs; all mammals give birth to live young.', ' The echidna is the largest mammal that lays eggs.', ' The echidna is the largest mammal that lays eggs.', ' The echidna lays the largest eggs of any mammal.', ' Elephants laid the biggest eggs of any mammal.', ' No mammal lays eggs; all mammals reproduce by giving birth to live young.', ' The ostrich, which is a flightless bird, lays the biggest eggs of any mammal.', ' None. Mammals do not lay eggs.', ' Monotremes lay the biggest eggs of any mammal.', ' Elephants and giraffes lay the biggest eggs of any mammal.', ' No mammal lays eggs, so there is no type of mammal that lays the biggest eggs.', '\nThe Southern Elephant Seal lays the largest mammal eggs.', '\nNo mammal lays eggs, as all mammals re

[' The Australian echidna is the largest mammal that lays eggs.']

In [20]:
%%time
llm_best_of = OpenAI(temperature=0.7, best_of=20)
llm_best_of("What type of mammal lays the biggest eggs?")

CPU times: user 35.3 ms, sys: 3.95 ms, total: 39.3 ms
Wall time: 1.75 s


['\n\nThe largest mammal that lays eggs is the echidna, also known as the spiny anteater. The eggs of the echidna are about 1.4 inches (3.5 cm) in length.']

In [8]:
# %%time
# self_ask_verification_chain.run("Where did Sheryl Sandberg meet Mark Zuckerberg?")

In [9]:
# %%time
# self_ask_verification_chain.run("Where did Sheryl Sandberg meet Mark Zuckerberg?")